In [14]:
import json
import os
from pathlib import Path
from src.os_path_management import adapt_path, check_os

# STEP 0: Open config.json
file_config_path = Path(os.path.join(os.getcwd(), 'config.json'))
assert file_config_path.exists(), "File config.json not found"
with open(file_config_path, "r", encoding="utf-8") as f:
    config = json.load(f)

DATA_DIR = adapt_path(config["data_dir"])                            # str: data/genres
DOWNLOAD_MISSING_DATASET = config["download_missing_dataset"]        # bool
FEATURES_DIR = adapt_path(config["features_dir"])                    # str: data/features
GENRES = config["genres"]                                            # list[str]
INSTALL_MISSING_LIBRARIES = config["install_missing_libraries"]      # bool
LIBRARIES = config["libraries"]                                      # list[str]
RECALCULATE_FEATURES_ALWAYS = config["recalculate_features_always"]  # bool

# STEP 2: Check the current working directory and validate it with an assertion
CHECK_OS = check_os()
SYSTEM_DETECTED = CHECK_OS[0]
assert SYSTEM_DETECTED, CHECK_OS[1]

# Check pwd directory
print("Actual directory: ", os.getcwd())
assert os.path.basename(os.getcwd()) == "GTZAN_Genre_Recognizer", "The script should be starded from root-project folder with GTZAN_Genre_Recognizer name"

script_path = os.path.join(os.getcwd(), "src")
script_download_missing_library = os.path.join(script_path, "download_missing_library.py")
script_download_data = os.path.join(script_path, "download_data.py")

FEATURES_FILE_NAME = "X_features.npy"
LABELS_FILE_NAME = "y_labels.npy"
FEATURES_PATH = os.path.join(FEATURES_DIR, FEATURES_FILE_NAME)
LABELS_PATH = os.path.join(FEATURES_DIR, LABELS_FILE_NAME)

# STEP 1: Check mandatory libraries to run the notebooks
print(f"Find install_missing_libraries = {INSTALL_MISSING_LIBRARIES} in config.json")
if INSTALL_MISSING_LIBRARIES:
    print(f"Missing libraries will be installed automatically")
else:
    print(f"Missing libraries will not be installed automatically")

for pkg in LIBRARIES:
    try:
        __import__(pkg)
    except ImportError:
        if INSTALL_MISSING_LIBRARIES:
            print(f"Installing {pkg}")
            !python "{script_download_missing_library}" {pkg}
        else:
            raise ImportError(f"Mandatory library not found: {pkg}."
                              f"Run the following command to install it: pip install {pkg} or set install_missing_libraries = true in config.json")

#DATASET_EXTENSION = "au"

# STEP 3: Run all the notebooks in sequence
##TODO: Test download_data.py and invoke it when dataset is not valid - test missing
%run notebooks/01_data_setup.ipynb    # Prepare Dataset

if RECALCULATE_FEATURES_ALWAYS or (not os.path.exists(FEATURES_PATH) and not os.path.exists(LABELS_PATH)): # Check configuration RECALCULATE_FEATURES_ALWAYS == True or if features already exist
    %run notebooks/02_features.ipynb  # Feature Extraction
else:
    print(f"Found {FEATURES_DIR}/{FEATURES_FILE_NAME} and {LABELS_FILE_NAME}, skipping notebooks/02_features.ipynb")

CROSS_VALIDATION_FOLDS_NUMBER = config["cross_validation_folds_number"]
RANDOM_STATE = config["random_state"]
%run notebooks/03_training.ipynb      # Training and Model Selection

%run notebooks/04_evaluation.ipynb    # Final Evaluation on Test Set

%run notebooks/05_inference.ipynb     # Inference on New Audio Files

Actual directory:  c:\Users\rpork\Desktop\Documenti\Studio\MAGISTRALE\Machine Learning\GTZAN_Genre_Recognizer
Find install_missing_libraries = True in config.json
Missing libraries will be installed automatically
Check if data exists at the following path:  data\genres
The dataset is incomplete, the dataset will be downloaded in a new version
All subfolders found
Number of tracks founds:
blues     : 0
The dataset is incomplete, the dataset will be downloaded in a new version


Traceback (most recent call last):
  File "c:\Users\rpork\Desktop\Documenti\Studio\MAGISTRALE\Machine Learning\GTZAN_Genre_Recognizer\src\download_data.py", line 13, in <module>
    from src.os_path_management import adapt_path
ModuleNotFoundError: No module named 'src'


classical : 0
The dataset is incomplete, the dataset will be downloaded in a new version


Traceback (most recent call last):
  File "c:\Users\rpork\Desktop\Documenti\Studio\MAGISTRALE\Machine Learning\GTZAN_Genre_Recognizer\src\download_data.py", line 13, in <module>
    from src.os_path_management import adapt_path
ModuleNotFoundError: No module named 'src'


country   : 0
The dataset is incomplete, the dataset will be downloaded in a new version


...

Traceback (most recent call last):
  File "c:\Users\rpork\Desktop\Documenti\Studio\MAGISTRALE\Machine Learning\GTZAN_Genre_Recognizer\src\download_data.py", line 13, in <module>
    from src.os_path_management import adapt_path
ModuleNotFoundError: No module named 'src'


Total tracks: 0
The dataset is incomplete, the dataset will be downloaded in a new version


Traceback (most recent call last):
  File "c:\Users\rpork\Desktop\Documenti\Studio\MAGISTRALE\Machine Learning\GTZAN_Genre_Recognizer\src\download_data.py", line 13, in <module>
    from src.os_path_management import adapt_path
ModuleNotFoundError: No module named 'src'
Traceback (most recent call last):
  File "c:\Users\rpork\Desktop\Documenti\Studio\MAGISTRALE\Machine Learning\GTZAN_Genre_Recognizer\src\download_data.py", line 13, in <module>
    from src.os_path_management import adapt_path
ModuleNotFoundError: No module named 'src'
Extracting features: 0it [00:00, ?it/s]


ValueError: need at least one array to concatenate


KeyboardInterrupt

